In [1]:
import pandas as pd
import pymer4.models
import numpy as np
import scipy.stats
import os

# Output and formatting settings

In [2]:
column_names = {"Estimate": "$\\beta$", "Z-stat": "$z$", "P-val": "$p$", "T-stat": "$t$", "F-stat": "$F$", "2.5_ci": "CI 2.5\%", "97.5_ci": "CI 97.5\%", "NumDF": "df"}
var_names = {"tta_z": "$\\textrm{TTA}$",
             "d_z": "distance",
             "time_budget_z": "time budget",
             "decision1": "decision",
             "dwell_mirror_z": "\% dwell time mirror",
             "RT_z": "RT",
             "tta_z:time_budget_z": "$\\textrm{TTA}$:time budget",
             "d_z:dwell_mirror_z": "distance:\% dwell time mirror",
             "tta_z:dwell_mirror_z": "$\\textrm{TTA}$:\% dwell time mirror",
             "time_budget_z:dwell_mirror_z": "time budget:\% dwell time mirror",
             "tta_z:time_budget_z:dwell_mirror_z": "$\\textrm{TTA}$:time budget:\% dwell time mirror",
             "decision1:d_z": "decision:distance",
             "decision1:tta_z": "decision:$\\textrm{TTA}$",
             "decision1:time_budget_z": "decision:time budget",
             "decision1:tta_z:time_budget_z": "decision:$\\textrm{TTA}$:time budget"}

def p_formatted(p):
    if p>0.01:
        return "{:.2f}".format(p)
    elif p>0.001:
        return "{:.3f}".format(p)
    else:
        return "$<0.001$"

# Read and pre-process the data

In [3]:
output_path = "C:\\Users\\azgonnikov\\Dropbox\\Apps\\Overleaf\\AAP Dynamics of merging decisions Elsevier template"

processed_data_path = "data/processed/" 
data = pd.read_csv(os.path.join(processed_data_path, "processed_eye_data.csv"))
metrics = pd.read_csv(os.path.join(processed_data_path, "metrics.csv"))

def get_z_score(x):
    return (x-x.mean())/x.std()

for col in ["RT", "d", "tta", "time_budget", "dwell_mirror"]:
    metrics.loc[:, col+"_z"] = get_z_score(metrics[col]) 

# Decision outcome as a function of kinematic conditions

In [4]:
model_decision = pymer4.models.Lmer("is_gap_accepted ~ 1 + d_z + tta_z + time_budget_z + (1 + tta_z + time_budget_z + d_z | participant)", data=metrics, family="binomial")
model_decision_fit = model_decision.fit(summarize=True)
model_decision.coefs

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: is_gap_accepted~1+d_z+tta_z+time_budget_z+(1+tta_z+time_budget_z+d_z|participant)

Family: binomial	 Inference: parametric

Number of observations: 9354	 Groups: {'participant': 26.0}

Log-likelihood: -3470.421 	 AIC: 6968.843

Random effects:

                      Name    Var    Std
participant    (Intercept)  1.628  1.276
participant          tta_z  0.264  0.514
participant  time_budget_z  0.216  0.465
participant            d_z  0.257  0.507

                       IV1            IV2   Corr
participant    (Intercept)          tta_z -0.190
participant    (Intercept)  time_budget_z -0.209
participant    (Intercept)            d_z  0.281
participant          tta_z  time_budget_z -0.198
participant          tta_z            d_z -0.596
participant  time_budget_z            d_z  0.450

Fixed effects:


,Estimate,2.5_ci,97.5_ci,SE,OR,OR_2.5_ci,OR_97.5_ci,Prob,Prob_2.5_ci,Prob_97.5_ci,Z-stat,P-val,Sig
(Intercept),1.235310,0.735969,1.734650,0.254770,3.439443,2.087504,5.666943,0.774747,0.676114,0.850006,4.848722,1.242594e-06,***
d_z,0.496873,0.291231,0.702515,0.104921,1.643573,1.338074,2.018823,0.621724,0.572297,0.668745,4.735676,2.183258e-06,***
tta_z,1.877860,1.659007,2.096714,0.111662,6.539498,5.254090,8.139379,0.867365,0.840105,0.890583,16.817361,1.820900e-63,***
time_budget_z,-0.454711,-0.644824,-0.264598,0.096998,0.634631,0.524755,0.767514,0.388241,0.344157,0.434234,-4.687834,2.761124e-06,***


In [5]:
coefs = model_decision.coefs.loc[:, ["Estimate", "SE", "Z-stat", "P-val"]]
coefs["P-val"] = coefs["P-val"].apply(p_formatted)
styler = coefs.rename(columns=column_names, index=var_names).style.format(precision=2)

with open(os.path.join(output_path, "tab_decision.tex"), 'w') as writer:
     writer.write(styler.to_latex(
         column_format="rrrrr", position="h", position_float="centering",
         hrules=True, label="tab:decision", caption="Standardized coefficients of the mixed-effects logistic regression describing the final decision. All effects were modelled as random slopes per participant: \\texttt{decision $\sim$ 1 + distance + TTA + time budget + (1 + distance + TTA + time budget) | participant}."
     )
)

# Response time

In [6]:
model_RT = pymer4.models.Lmer("RT_z ~ 1 + decision*(d_z + tta_z*time_budget_z) + (decision | participant) ", data=metrics, family="gaussian")
model_RT.fit(summarize=True, factors={"decision": ["Accept", "Reject"]})
model_RT.coefs

Linear mixed model fit by REML [’lmerMod’]
Formula: RT_z~1+decision*(d_z+tta_z*time_budget_z)+(decision|participant)

Family: gaussian	 Inference: parametric

Number of observations: 9354	 Groups: {'participant': 26.0}

Log-likelihood: -10000.986 	 AIC: 20029.971

Random effects:

                       Name    Var    Std
participant     (Intercept)  0.332  0.576
participant  decisionReject  0.213  0.462
Residual                     0.482  0.694

                     IV1             IV2   Corr
participant  (Intercept)  decisionReject -0.419

Fixed effects:


,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val,Sig
(Intercept),-0.264038,-0.486344,-0.041732,0.113423,25.107009,-2.327897,2.826776e-02,*
decision1,1.071699,0.888574,1.254823,0.093433,26.061084,11.470294,1.098433e-11,***
d_z,-0.035816,-0.053540,-0.018092,0.009043,9295.998708,-3.960684,7.529327e-05,***
tta_z,0.043968,0.023941,0.063995,0.010218,9307.683628,4.302981,1.702384e-05,***
time_budget_z,0.064291,0.044864,0.083719,0.009912,9296.496516,6.486153,9.254174e-11,***
tta_z:time_budget_z,0.045797,0.026404,0.065191,0.009895,9295.556832,4.628444,3.734136e-06,***
decision1:d_z,0.087718,0.057324,0.118111,0.015507,9307.775132,5.656655,1.589144e-08,***
decision1:tta_z,0.283923,0.242029,0.325817,0.021375,9316.166689,13.283008,6.690803e-40,***
decision1:time_budget_z,0.125070,0.084361,0.165778,0.020770,9299.355557,6.021598,1.792446e-09,***
decision1:tta_z:time_budget_z,0.039901,-0.000596,0.080398,0.020662,9297.470195,1.931113,5.349950e-02,.


In [7]:
coefs = model_RT.coefs.loc[:, ["Estimate", "SE", "T-stat", "P-val"]]
coefs["P-val"] = coefs["P-val"].apply(p_formatted)
styler = coefs.rename(columns=column_names, index=var_names).style.format(precision=2)

with open(os.path.join(output_path, "tab_RT.tex"), 'w') as writer:
     writer.write(styler.to_latex(
         column_format="rrrrr", position="h", position_float="centering",
         hrules=True, label="tab:RT", caption="Standardized coefficients of the mixed-effects linear regression describing response times. Random slope of decision was included per participant: \\texttt{RT $\sim$ 1 + decision*(TTA*time budget + distance) + (1 + decision) | participant}. ``Accept'' was set as a reference level for the decision outcome factor."
     )
)

## ANOVA

In [8]:
RT_anova = model_RT.anova()
RT_decision_marginal_estimates, RT_decision_comparisons = model_RT.post_hoc(marginal_vars=["decision"])
RT_anova

SS Type III Analysis of Variance Table with Satterthwaite approximated degrees of freedom:
(NOTE: Using original model contrasts, orthogonality not guaranteed)


,SS,MS,NumDF,DenomDF,F-stat,P-val,Sig
decision,63.432596,63.432596,1,26.061084,131.567652,1.098433e-11,***
d_z,0.518863,0.518863,1,9310.895838,1.076190,2.995784e-01,
tta_z,145.871728,145.871728,1,9313.506604,302.557390,1.028326e-66,***
time_budget_z,71.921823,71.921823,1,9303.012663,149.175438,4.781966e-34,***
tta_z:time_budget_z,19.527744,19.527744,1,9298.171011,40.503142,2.055678e-10,***
decision:d_z,15.427044,15.427044,1,9307.775132,31.997744,1.589144e-08,***
decision:tta_z,85.066039,85.066039,1,9316.166689,176.438294,6.690803e-40,***
decision:time_budget_z,17.481832,17.481832,1,9299.355557,36.259647,1.792446e-09,***
decision:tta_z:time_budget_z,1.797954,1.797954,1,9297.470195,3.729196,5.349950e-02,.


In [9]:
coefs = RT_anova.loc[:, ["SS", "MS", "F-stat", "P-val"]]
coefs["P-val"] = coefs["P-val"].apply(p_formatted)
styler = coefs.rename(columns=column_names, index=var_names).style.format(precision=2)

with open(os.path.join(output_path, "tab_RT_ANOVA.tex"), 'w') as writer:
     writer.write(styler.to_latex(
         column_format="rrrrrr", position="h", position_float="centering",
         hrules=True, label="tab:RT_ANOVA", caption="ANOVA table based on the mixed-effects linear regression describing response time. Random slope of decision was included per participant: \\texttt{RT $\sim$ 1 + decision*(TTA*time budget + distance) + (1 + decision) | participant}."
     )
)

## Difference between accept and reject RTs

In [10]:
RT_decision_marginal_estimates, RT_decision_comparisons = model_RT.post_hoc(marginal_vars=["decision"])
RT_decision_comparisons

,Contrast,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val,Sig
1,Accept - Reject,-1.072,-1.264,-0.88,0.093,26.061,-11.471,0.0,***


In [11]:
RT_decision_comparisons.Estimate*metrics.RT.std()

1   -0.667688
Name: Estimate, dtype: float64

## Estimates of condition effects on RT per decision

In [12]:
def get_marginal_estimates(model_RT, marginal_vars):
    marginal_estimates, comparisons = model_RT.post_hoc(marginal_vars=marginal_vars, grouping_vars=["decision"])
    marginal_estimates["T-stat"] = marginal_estimates["Estimate"]/marginal_estimates["SE"]
    marginal_estimates["P-val"] = scipy.stats.t.sf(np.abs(marginal_estimates["T-stat"]), marginal_estimates.DF)
    return marginal_estimates

In [13]:
get_marginal_estimates(model_RT, "tta_z")

,decision,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val
1,Accept,0.044,0.024,0.064,0.010,9307.682,4.400000,5.472812e-06
2,Reject,0.328,0.291,0.365,0.019,9315.358,17.263158,4.669593e-66


In [14]:
get_marginal_estimates(model_RT, "d_z")

,decision,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val
1,Accept,-0.036,-0.054,-0.018,0.009,9295.999,-4.0,0.000032
2,Reject,0.052,0.027,0.077,0.013,9313.936,4.0,0.000032


In [15]:
get_marginal_estimates(model_RT, "time_budget_z")

,decision,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val
1,Accept,0.064,0.045,0.084,0.010,9296.497,6.4,8.144008e-11
2,Reject,0.189,0.154,0.225,0.018,9302.492,10.5,6.007415e-26


## Estimates of time budget effect per decision and TTA level

In [16]:
marginal_estimates, comparisons = model_RT.post_hoc(marginal_vars=["time_budget_z"], grouping_vars=["decision", "tta_z"])
marginal_estimates["T-stat"] = marginal_estimates["Estimate"]/marginal_estimates["SE"]
marginal_estimates["P-val"] = scipy.stats.t.sf(np.abs(marginal_estimates["T-stat"]), marginal_estimates.DF)

In [17]:
marginal_estimates

,decision,tta_z,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val
1,Accept,-1.0,0.018,-0.014,0.051,0.017,9296.700,1.058824,1.448538e-01
2,Reject,-1.0,0.104,0.077,0.130,0.013,9307.115,8.000000,6.964658e-16
3,Accept,1.0,0.110,0.089,0.131,0.011,9294.359,10.000000,1.000565e-23
4,Reject,1.0,0.275,0.209,0.341,0.034,9299.382,8.088235,3.405848e-16


# Decision outcome as a function of dwell time and response time

In [18]:
model_decision_dwell_RT = pymer4.models.Lmer("is_gap_accepted ~ (d_z + tta_z + time_budget_z)*dwell_mirror_z + RT_z + (tta_z + time_budget_z | participant) ", data=metrics, family="binomial")
model_decision_dwell_RT_fit = model_decision_dwell_RT.fit(summarize=True)
model_decision_dwell_RT.coefs

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: is_gap_accepted~(d_z+tta_z+time_budget_z)*dwell_mirror_z+RT_z+(tta_z+time_budget_z|participant)

Family: binomial	 Inference: parametric

Number of observations: 9354	 Groups: {'participant': 26.0}

Log-likelihood: -2697.486 	 AIC: 5424.971

Random effects:

                      Name    Var    Std
participant    (Intercept)  2.639  1.625
participant          tta_z  0.623  0.789
participant  time_budget_z  0.125  0.353

                     IV1            IV2   Corr
participant  (Intercept)          tta_z  0.390
participant  (Intercept)  time_budget_z -0.515
participant        tta_z  time_budget_z -0.293

Fixed effects:


,Estimate,2.5_ci,97.5_ci,SE,OR,OR_2.5_ci,OR_97.5_ci,Prob,Prob_2.5_ci,Prob_97.5_ci,Z-stat,P-val,Sig
(Intercept),1.632110,0.995442,2.268778,0.324837,5.114655,2.705920,9.667577,0.836458,0.730161,0.906258,5.024404,5.049976e-07,***
d_z,0.474443,0.403363,0.545522,0.036265,1.607118,1.496851,1.725508,0.616435,0.599496,0.633096,13.082479,4.146934e-39,***
tta_z,2.253207,1.924368,2.582046,0.167778,9.518213,6.850817,13.224172,0.904927,0.872625,0.929697,13.429679,4.051617e-41,***
time_budget_z,-0.305540,-0.460844,-0.150235,0.079238,0.736726,0.630751,0.860505,0.424204,0.386786,0.462512,-3.855955,1.152789e-04,***
dwell_mirror_z,-0.338802,-0.455872,-0.221732,0.059731,0.712624,0.633895,0.801130,0.416101,0.387965,0.444793,-5.672144,1.410213e-08,***
RT_z,-1.919034,-2.035556,-1.802513,0.059451,0.146749,0.130608,0.164884,0.127969,0.115520,0.141545,-32.279418,1.360434e-228,***
d_z:dwell_mirror_z,-0.046112,-0.119143,0.026919,0.037261,0.954935,0.887681,1.027284,0.488474,0.470249,0.506729,-1.237531,2.158899e-01,
tta_z:dwell_mirror_z,0.148806,0.034332,0.263279,0.058406,1.160448,1.034928,1.301190,0.537133,0.508582,0.565442,2.547782,1.084103e-02,*
time_budget_z:dwell_mirror_z,0.217606,0.124588,0.310624,0.047459,1.243097,1.132681,1.364277,0.554188,0.531107,0.577038,4.585118,4.537309e-06,***


In [19]:
coefs = model_decision_dwell_RT.coefs.loc[:, ["Estimate", "SE", "Z-stat", "P-val"]]
coefs["P-val"] = coefs["P-val"].apply(p_formatted)
styler = coefs.rename(columns=column_names, index=var_names).style.format(precision=2)

with open(os.path.join(output_path, "tab_decision_dwell_RT.tex"), 'w') as writer:
     writer.write(styler.to_latex(
         column_format="rrrrr", position="!ht", position_float="centering",
         hrules=True, label="tab:decision_dwell_RT", caption="Standardized coefficients of the mixed-effects logistic regression describing the decision outcome as a function of kinematic variables, response time, and relative dwell time. Random slopes of TTA to the overtaking vehicle and the time budget provided by the merging lane were included per participant: \\texttt{decision $\sim$ 1 + RT + distance + (TTA * time budget * \\% dwell time mirror) + (1 + TTA + time budget) | participant}."
     )
)